<a href="https://colab.research.google.com/github/yannickfunk/EmbeddedMLLab/blob/main/end2end.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys

IN_COLAB = False

# if in colab, make colab setup
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')

    IN_COLAB = True
    ZIP_PATH = "/content/drive/MyDrive/embedded_ml_data/VOCdevkit.zip"
    FILENAME = "end2end.ipynb"
    BRANCH = "main"
    !git clone -b $BRANCH https://github.com/yannickfunk/EmbeddedMLLab tmp
    !rm tmp/$FILENAME
    !mv tmp/* .
    !rm -rf tmp
    !rm -rf sample_data
    !pip install -r requirements.txt

    # setup data
    !cp  $ZIP_PATH data/
    %pushd data
    !unzip -qq VOCdevkit.zip
    %popd

Mounted at /content/drive
Cloning into 'tmp'...
remote: Enumerating objects: 63, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 63 (delta 8), reused 17 (delta 5), pack-reused 34
Unpacking objects: 100% (63/63), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 4.5 MB/s 
     |████████████████████████████████| 128 kB 74.5 MB/s 
     |████████████████████████████████| 512 kB 73.0 MB/s 
     |████████████████████████████████| 125 kB 71.5 MB/s 
     |████████████████████████████████| 72 kB 1.9 MB/s 
     |████████████████████████████████| 66 kB 6.6 MB/s 
     |████████████████████████████████| 373 kB 72.9 MB/s 
     |████████████████████████████████| 58 kB 7.6 MB/s 
     |████████████████████████████████| 55 kB 4.2 MB/s 
     |████████████████████████████████| 237 kB 79.5 MB/s 
     |████████████████████████████████| 55 kB

In [6]:
import os
import torch
GPU_AVAILABLE = torch.cuda.is_available()
DATA_PATH = "data/"
DOWNLOAD = False

print("Using GPU:", GPU_AVAILABLE)

Using GPU: True


In [8]:
from models.tinyyolov2 import TinyYoloV2Original
from utils.dataloader import VOCDataLoaderPerson
import lightning as pl

train_loader = VOCDataLoaderPerson(train=True, batch_size=128, shuffle=True, data_path=DATA_PATH, download=DOWNLOAD)
val_loader = VOCDataLoaderPerson(train=False, batch_size=128, data_path=DATA_PATH, download=DOWNLOAD)

model = TinyYoloV2Original(num_classes=1)
model.load_pt_from_disk("data/voc_pretrained.pt")
trainer = pl.Trainer(max_epochs=15, accelerator='gpu' if GPU_AVAILABLE else 'cpu', log_every_n_steps=10, limit_val_batches=350)

trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader)

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
   | Name  | Type            | Params
-------------------------------------------
0  | loss  | YoloLoss        | 0     
1  | pad   | ReflectionPad2d | 0     
2  | conv1 | Conv2d          | 432   
3  | bn1   | BatchNorm2d     | 32    
4  | conv2 | Conv2d          | 4.6 K 
5  | bn2   | BatchNorm2d     | 64    
6  | conv3 | Conv2d          | 18.4 K
7  | bn3   | B

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]